# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [205]:
import graphlab

In [206]:
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [207]:
sales = graphlab.SFrame('data/kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [208]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [209]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [210]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              solver='fista',l1_penalty=1e10,l2_penalty=0.,
                                              validation_set=None, verbose=False)

Find what features had non-zero weight.

In [211]:
model_all['coefficients']['value'].nnz()

6

In [212]:
model_all.get('coefficients').print_rows(num_rows=18, num_columns=3)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [213]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [214]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [263]:
RSS = []
for p in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features, solver='fista',
                                              l1_penalty=p, l2_penalty=0., 
                                              validation_set=None, verbose=False)
    
    pred = model.predict(validation)
    nnz = model['coefficients']['value'].nnz()
    rss = (sum(validation['price'] - pred)**2) + p*sum(abs(model.get('coefficients')['value'][1:]))
    print "l1_penalty : %f gives, %f Validation Error & %i Non zeros"%(p, rss, nnz)
    RSS.append(rss)

l1_penalty : 10.000000 gives, 22076011653144548.000000 Validation Error & 18 Non zeros
l1_penalty : 31.622777 gives, 22076008993398704.000000 Validation Error & 18 Non zeros
l1_penalty : 100.000000 gives, 22076000582545140.000000 Validation Error & 18 Non zeros
l1_penalty : 316.227766 gives, 22075973985103296.000000 Validation Error & 18 Non zeros
l1_penalty : 1000.000000 gives, 22075889876706732.000000 Validation Error & 18 Non zeros
l1_penalty : 3162.277660 gives, 22075623903684160.000000 Validation Error & 18 Non zeros
l1_penalty : 10000.000000 gives, 22074782833760856.000000 Validation Error & 18 Non zeros
l1_penalty : 31622.776602 gives, 22072123243691924.000000 Validation Error & 18 Non zeros
l1_penalty : 100000.000000 gives, 22063713948046648.000000 Validation Error & 18 Non zeros
l1_penalty : 316227.766017 gives, 22037132080444328.000000 Validation Error & 18 Non zeros
l1_penalty : 1000000.000000 gives, 21953179465493176.000000 Validation Error & 18 Non zeros
l1_penalty : 31622

In [264]:
min(RSS)

20930880157874812.0

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [265]:
model = graphlab.linear_regression.create(training, target='price', 
                                          features=all_features, solver='fista',
                                          l2_penalty=0., l1_penalty=10000000,
                                          validation_set=None, verbose=False)
test_pred = model.predict(testing)
print "Error on the test set : ",(((testing['price'] - test_pred)**2).sum())

Error on the test set :  1.56972779669e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [218]:
model['coefficients']['value'].nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [219]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [220]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [221]:
NNZ = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features,
                                              l2_penalty=0., l1_penalty=l1_penalty,
                                              validation_set=None, verbose=False)
    
    non_zeros = model.get('coefficients')['value'].nnz()
    print "For the penalty: %f => We get  %i "%(l1_penalty, non_zeros)

For the penalty: 100000000.000000 => We get  18 
For the penalty: 127427498.570313 => We get  18 
For the penalty: 162377673.918872 => We get  18 
For the penalty: 206913808.111479 => We get  18 
For the penalty: 263665089.873036 => We get  17 
For the penalty: 335981828.628379 => We get  17 
For the penalty: 428133239.871940 => We get  17 
For the penalty: 545559478.116851 => We get  17 
For the penalty: 695192796.177559 => We get  17 
For the penalty: 885866790.410083 => We get  16 
For the penalty: 1128837891.684688 => We get  15 
For the penalty: 1438449888.287666 => We get  15 
For the penalty: 1832980710.832438 => We get  13 
For the penalty: 2335721469.090121 => We get  12 
For the penalty: 2976351441.631313 => We get  10 
For the penalty: 3792690190.732254 => We get  6 
For the penalty: 4832930238.571753 => We get  5 
For the penalty: 6158482110.660254 => We get  3 
For the penalty: 7847599703.514623 => We get  1 
For the penalty: 10000000000.000000 => We get  1 


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [222]:
l1_penalty_min = 2976351441.631313
l1_penalty_max = 3792690190.732254

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [223]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [270]:
RSS = []
for l1_penalty in l1_penalty_values:
    
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features, solver='fista',
                                              l1_penalty=l1_penalty, l2_penalty=0., 
                                              validation_set=None, verbose=False)
    
    pred = model.predict(validation)
    nnz = model['coefficients']['value'].nnz()
    rss = (sum(validation['price'] - pred)**2) + l1_penalty*sum(abs(model.get('coefficients')['value'][1:]))
    print "l1_penalty : %f gives, %f Validation Error & %i Non zeros"%(l1_penalty, rss, nnz)
    RSS.append(rss)

l1_penalty : 2976351441.631313 gives, 799633399392109824.000000 Validation Error & 10 Non zeros
l1_penalty : 3019316638.952415 gives, 814603074144637056.000000 Validation Error & 10 Non zeros
l1_penalty : 3062281836.273517 gives, 829710225162469760.000000 Validation Error & 10 Non zeros
l1_penalty : 3105247033.594619 gives, 850953077985146496.000000 Validation Error & 10 Non zeros
l1_penalty : 3148212230.915721 gives, 880725991645469568.000000 Validation Error & 10 Non zeros
l1_penalty : 3191177428.236824 gives, 911789648908262528.000000 Validation Error & 10 Non zeros
l1_penalty : 3234142625.557926 gives, 943391765605216128.000000 Validation Error & 10 Non zeros
l1_penalty : 3277107822.879028 gives, 975532148689522432.000000 Validation Error & 10 Non zeros
l1_penalty : 3320073020.200130 gives, 996181489639752576.000000 Validation Error & 8 Non zeros
l1_penalty : 3363038217.521232 gives, 1011332904045491072.000000 Validation Error & 8 Non zeros
l1_penalty : 3406003414.842335 gives, 103

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [267]:
min(RSS)

7.9963339939210982e+17

In [248]:
model['coefficients'].print_rows(num_rows=18, num_columns=3)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  240309.75932 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 13840.6399577 |
|   sqft_living    |  None | 30.5583588298 |
| sqft_living_sqrt |  None | 592.199469213 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 2265.12052556 |
|    sqft_above    |  None | 27.4878726568 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x